In [ ]:
!pip install -q torch_geometric
!pip install -q torch_scatter

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import Parameter, Linear, BatchNorm1d
from torch_scatter import scatter_add
from torch_geometric.nn import MessagePassing, GATConv, global_add_pool
from torch_geometric.utils import add_self_loops, remove_self_loops
from torch_geometric.data import Data
from torch_geometric.nn.inits import glorot, zeros

import gc
import json
import copy
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import partial

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, balanced_accuracy_score, confusion_matrix
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(42)

# **GitHub**
https://github.com/yongduosui/CAL/tree/main

In [ ]:
class GCNConv(MessagePassing):
    
    def __init__(self,
                 in_channels,
                 out_channels,
                 improved=False,
                 cached=False,
                 bias=True,
                 edge_norm=True,
                 gfn=False):
        super(GCNConv, self).__init__('add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.edge_norm = edge_norm
        self.gfn = gfn
        self.message_mask = None
        self.weight = Parameter(torch.Tensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        self.cached_result = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, improved=False, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ),
                                     dtype=dtype,
                                     device=edge_index.device)
        
        edge_weight = edge_weight.view(-1)
        
        
        assert edge_weight.size(0) == edge_index.size(1)
        
        edge_index, edge_weight = remove_self_loops(edge_index, edge_weight)
        edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)
        # Add edge_weight for loop edges.
        loop_weight = torch.full((num_nodes, ),
                                 1 if not improved else 2,
                                 dtype=edge_weight.dtype,
                                 device=edge_weight.device)
        edge_weight = torch.cat([edge_weight, loop_weight], dim=0)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        
        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight=None):
        """"""
        
        x = torch.matmul(x, self.weight)
        if self.gfn:
            return x
    
        if not self.cached or self.cached_result is None:
            if self.edge_norm:
                edge_index, norm = GCNConv.norm(
                    edge_index, 
                    x.size(0), 
                    edge_weight, 
                    self.improved, 
                    x.dtype)
            else:
                norm = None
            self.cached_result = edge_index, norm

        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):

        if self.edge_norm:
            return norm.view(-1, 1) * x_j
        else:
            return x_j
        
    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [ ]:
class CausalGAT(torch.nn.Module):
    def __init__(self, num_features,
                       num_classes,
                       tokens_dim=768,
                       hidden_dim=256,
                       num_conv_layers=3,
                       fusion_mode=None,
                       head=4, 
                       dropout=0.3):
        super(CausalGAT, self).__init__()
        self.cat_or_add = "add"
        self.fusion_mode = fusion_mode
        self.global_pool = global_add_pool
        self.dropout = dropout
        GConv = partial(GCNConv, edge_norm=True, gfn=False)

        hidden_in = num_features
        self.num_classes = num_classes
        hidden_out = num_classes
        self.fc_num = 222

        if self.fusion_mode == 'concat':
            total_input_dim = num_features + tokens_dim
            self.concat_proj = Linear(total_input_dim, hidden_dim)
        elif self.fusion_mode == 'gated':
            self.feat_proj_gate = Linear(num_features, hidden_dim)
            self.code_proj_gate = Linear(tokens_dim, hidden_dim)
            self.gate_linear = Linear(num_features + tokens_dim, hidden_dim)
        elif self.fusion_mode == 'cross_atten':
            self.q_proj = Linear(num_features, hidden_dim)
            self.k_proj = Linear(tokens_dim, hidden_dim)
            self.v_proj = Linear(tokens_dim, hidden_dim)
            self.cross_attn = nn.MultiheadAttention(
                embed_dim=hidden_dim, 
                num_heads=head, 
                batch_first=True
            )

        if self.fusion_mode is not None:
            self.bn_feat = BatchNorm1d(hidden_dim)
            self.conv_feat = GCNConv(hidden_dim, hidden_dim, gfn=True)
        else:
            self.bn_feat = BatchNorm1d(hidden_in)
            self.conv_feat = GCNConv(hidden_in, hidden_dim, gfn=True)
        
        self.bns_conv = torch.nn.ModuleList()
        self.convs = torch.nn.ModuleList()

        for i in range(num_conv_layers):
            self.bns_conv.append(BatchNorm1d(hidden_dim))
            self.convs.append(GATConv(hidden_dim, int(hidden_dim / head), heads=head, dropout=dropout))

        self.edge_att_mlp = Linear(hidden_dim * 2, 2)
        self.node_att_mlp = Linear(hidden_dim, 2)
        self.bnc = BatchNorm1d(hidden_dim)
        self.bno= BatchNorm1d(hidden_dim)
        self.context_convs = GConv(hidden_dim, hidden_dim)
        self.objects_convs = GConv(hidden_dim, hidden_dim)

        # context mlp
        self.fc1_bn_c = BatchNorm1d(hidden_dim)
        self.fc1_c = Linear(hidden_dim, hidden_dim)
        self.fc2_bn_c = BatchNorm1d(hidden_dim)
        self.fc2_c = Linear(hidden_dim, hidden_out)
        # object mlp
        self.fc1_bn_o = BatchNorm1d(hidden_dim)
        self.fc1_o = Linear(hidden_dim, hidden_dim)
        self.fc2_bn_o = BatchNorm1d(hidden_dim)
        self.fc2_o = Linear(hidden_dim, hidden_out)
        # random mlp
        if self.cat_or_add == "cat":
            self.fc1_bn_co = BatchNorm1d(hidden_dim * 2)
            self.fc1_co = Linear(hidden_dim * 2, hidden_dim)
            self.fc2_bn_co = BatchNorm1d(hidden_dim)
            self.fc2_co = Linear(hidden_dim, hidden_out)

        elif self.cat_or_add == "add":
            self.fc1_bn_co = BatchNorm1d(hidden_dim)
            self.fc1_co = Linear(hidden_dim, hidden_dim)
            self.fc2_bn_co = BatchNorm1d(hidden_dim)
            self.fc2_co = Linear(hidden_dim, hidden_out)
        else:
            assert False
        
        # BN initialization.
        for m in self.modules():
            if isinstance(m, (torch.nn.BatchNorm1d)):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0.0001)

    def forward(self, data, eval_random=True):

        x_features = data.x if data.x is not None else data.feat
        x_tokens = data.tokens if data.tokens is not None else data.feat
        edge_index, batch = data.edge_index, data.batch
        row, col = edge_index

        if self.fusion_mode == 'concat':
            x_concat = torch.cat([x_features, x_tokens], dim=-1)
            x = self.concat_proj(x_concat)
        elif self.fusion_mode == 'gated':
            x_feat_proj = self.feat_proj_gate(x_features)
            x_code_proj = self.code_proj_gate(x_tokens)
            gate_input = torch.cat([x_features, x_tokens], dim=-1)
            g = torch.sigmoid(self.gate_linear(gate_input))
            x = g * x_feat_proj + (1.0 - g) * x_code_proj
        elif self.fusion_mode == 'cross_atten':
            q = self.q_proj(x_features).unsqueeze(1)
            k = self.k_proj(x_tokens).unsqueeze(1)
            v = self.v_proj(x_tokens).unsqueeze(1)
            attn_output, _ = self.cross_attn(q, k, v)
            x = attn_output.squeeze(1)
        else:
            x = x_features
        
        x = self.bn_feat(x)
        x = F.relu(self.conv_feat(x, edge_index))
        
        for i, conv in enumerate(self.convs):
            x = self.bns_conv[i](x)
            x = F.relu(conv(x, edge_index))
        
        edge_rep = torch.cat([x[row], x[col]], dim=-1)
        edge_att = F.softmax(self.edge_att_mlp(edge_rep), dim=-1)
        edge_weight_c = edge_att[:, 0]
        edge_weight_o = edge_att[:, 1]

        node_att = F.softmax(self.node_att_mlp(x), dim=-1)
        xc = node_att[:, 0].view(-1, 1) * x
        xo = node_att[:, 1].view(-1, 1) * x
        xc = F.relu(self.context_convs(self.bnc(xc), edge_index, edge_weight_c))
        xo = F.relu(self.objects_convs(self.bno(xo), edge_index, edge_weight_o))

        xc = self.global_pool(xc, batch)
        xo = self.global_pool(xo, batch)
        
        xc_logis = self.context_readout_layer(xc)
        xo_logis = self.objects_readout_layer(xo)
        xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)
        return xc_logis, xo_logis, xco_logis

    def context_readout_layer(self, x):
        
        x = self.fc1_bn_c(x)
        x = self.fc1_c(x)
        x = F.relu(x)
        x = self.fc2_bn_c(x)
        x = self.fc2_c(x)
        x_logis = F.log_softmax(x, dim=-1)
        return x_logis

    def objects_readout_layer(self, x):
   
        x = self.fc1_bn_o(x)
        x = self.fc1_o(x)
        x = F.relu(x)
        x = self.fc2_bn_o(x)
        x = self.fc2_o(x)
        x_logis = F.log_softmax(x, dim=-1)
        return x_logis

    def random_readout_layer(self, xc, xo, eval_random):

        num = xc.shape[0]
        l = [i for i in range(num)]
        if eval_random:
            random.shuffle(l)
        random_idx = torch.tensor(l)
        
        if self.cat_or_add == "cat":
            x = torch.cat((xc[random_idx], xo), dim=1)
        else:
            x = xc[random_idx] + xo

        x = self.fc1_bn_co(x)
        x = self.fc1_co(x)
        x = F.relu(x)
        x = self.fc2_bn_co(x)
        x = self.fc2_co(x)
        x_logis = F.log_softmax(x, dim=-1)
        return x_logis

In [ ]:
def calculate_class_weights(label_list, device):
    """
    Calculate class weights for imbalanced dataset
    Returns:
        Tensor of shape [num_classes] with class weights
    """
    labels_tensor = torch.tensor([l.item() if isinstance(l, torch.Tensor) else l for l in label_list])
    class_counts = torch.bincount(labels_tensor)
    total = class_counts.sum().float()
    class_weights = total / (class_counts.float() + 1e-8)
    class_weights = class_weights / class_weights.sum() * len(class_counts)  # Normalize
    return class_weights.to(device)

In [ ]:
def load_parallel(codelm_embedd_path, graph_embedd_path, adjmat_path, meta_path, device):
    codelm_emb_tensor_all = torch.load(codelm_embedd_path)
    graph_emb_tensor_all = torch.load(graph_embedd_path)
    adj_tensor_all = torch.load(adjmat_path)

    with open(meta_path, 'r') as f:
        meta_data = json.load(f)

    labels_all = meta_data['labels']
    partitions = meta_data['partition']

    train_idx  = [i for i, p in enumerate(partitions) if p == 'train']
    valid_idx  = [i for i, p in enumerate(partitions) if p == 'valid']
    test_idx  = [i for i, p in enumerate(partitions) if p == 'test']
    
    def select_indices(indices):
        return (
            [codelm_emb_tensor_all[i].to(device) for i in indices],
            [graph_emb_tensor_all[i].to(device) for i in indices],
            [adj_tensor_all[i].to(device) for i in indices],
            [torch.tensor(labels_all[i], dtype=torch.long).numpy() for i in indices]
        )

    def shuffle_dataset(embedd_list, adj_list, label_list, indices):
        combined = list(zip(embedd_list, adj_list, label_list, indices))
        random.shuffle(combined)
        shuffled_embedd_list, shuffled_adj_list, shuffled_label_list, shuffled_indices = zip(*combined)
        return (
            list(shuffled_embedd_list),
            list(shuffled_adj_list),
            list(shuffled_label_list),
            list(shuffled_indices)
        )

    if len(train_idx) > 0:
        train_codelm_emb, train_graph_emb, train_adj, train_lbl = select_indices(train_idx)
        train_codelm_emb, train_graph_emb, train_adj, train_lbl = shuffle_dataset(train_codelm_emb, train_graph_emb, train_adj, train_lbl)
    else:
        train_codelm_emb, train_graph_emb, train_adj, train_lbl = [], [], [], []

    if len(valid_idx) > 0:
        valid_codelm_emb, valid_graph_emb, valid_adj, valid_lbl = select_indices(valid_idx)
        valid_codelm_emb, valid_graph_emb, valid_adj, valid_lbl = shuffle_dataset(valid_codelm_emb, valid_graph_emb, valid_adj, valid_lbl)
    else:
        valid_codelm_emb, valid_graph_emb, valid_adj, valid_lbl = [], [], [], []
    
    if len(test_idx) > 0:
        test_codelm_emb, test_graph_emb, test_adj, test_lbl = select_indices(test_idx)
        test_codelm_emb, test_graph_emb, test_adj, test_lbl = shuffle_dataset(test_codelm_emb, test_graph_emb, test_adj, test_lbl)
    else:
        test_codelm_emb, test_graph_emb, test_adj, test_lbl = [], [], [], []
    
    return {
        'train': (train_codelm_emb, train_graph_emb, train_adj, train_lbl),
        'valid': (valid_codelm_emb, valid_graph_emb, valid_adj, valid_lbl),
        'test': (test_codelm_emb, test_graph_emb, test_adj, test_lbl)
    }

In [ ]:
def merge_adversarial_dataset(original_test, adversarial_test):
    codelm_embedd_test, graph_embedd_test, adjmat_test, label_test = original_test
    adv_codelm_embedd_test, adv_graph_embedd_test, adv_adjmat_test, adv_label_test = adversarial_test

    # Split non-vulnerable samples
    nonvul_codelm, nonvul_graph, nonvul_adj, nonvul_lbl = [], [], [], []
    for emb_c, emb_g, adj, lbl in zip(codelm_embedd_test, graph_embedd_test, adjmat_test, label_test):
        if int(lbl) != 1:
            nonvul_codelm.append(emb_c)
            nonvul_graph.append(emb_g)
            nonvul_adj.append(adj)
            nonvul_lbl.append(lbl)

    print(f"🧩 Non-vulnerable samples: {len(nonvul_lbl)}")
    print(f"⚔️  Adversarial samples to add: {len(adv_label_test)}")

    # Merge dataset
    merged_codelm = nonvul_codelm + list(adv_codelm_embedd_test)
    merged_graph = nonvul_graph + list(adv_graph_embedd_test)
    merged_adj = nonvul_adj + list(adv_adjmat_test)
    merged_lbl = nonvul_lbl + list(adv_label_test)

    # Shuffle dataset
    combined = list(zip(merged_codelm, merged_graph, merged_adj, merged_lbl))
    random.shuffle(combined)
    sh_codelm, sh_graph, sh_adj, sh_lbl = zip(*combined)

    return list(sh_codelm), list(sh_graph), list(sh_adj), list(sh_lbl)

In [ ]:
# Setup device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
train_benign1_dataset = load_parallel(codelm_embedd_path="/kaggle/input/primevul-vuln-data/train_benign_part1/aggregated_codet5_embeddings.pt",
                                      graph_embedd_path="/kaggle/input/primevul-vuln-data/train_benign_part1/n2v_node_embeddings.pt",
                                      adjmat_path="/kaggle/input/primevul-vuln-data/train_benign_part1/adj_matrices.pt",
                                      meta_path="/kaggle/input/primevul-vuln-data/train_benign_part1/AlUNed.json",
                                      device=device)

In [ ]:
train_benign2_dataset = load_parallel(codelm_embedd_path="/kaggle/input/primevul-vuln-data/train_benign_part2/aggregated_codet5_embeddings.pt",
                                      graph_embedd_path="/kaggle/input/primevul-vuln-data/train_benign_part2/n2v_node_embeddings.pt",
                                      adjmat_path="/kaggle/input/primevul-vuln-data/train_benign_part2/adj_matrices.pt",
                                      meta_path="/kaggle/input/primevul-vuln-data/train_benign_part2/AlUNed.json",
                                      device=device)

In [ ]:
train_vulnerable_dataset = load_parallel(codelm_embedd_path="/kaggle/input/primevul-vuln-data/train_vuln/aggregated_codet5_embeddings.pt",
                                         graph_embedd_path="/kaggle/input/primevul-vuln-data/train_vuln/n2v_node_embeddings.pt",
                                         adjmat_path="/kaggle/input/primevul-vuln-data/train_vuln/adj_matrices.pt",
                                         meta_path="/kaggle/input/primevul-vuln-data/train_vuln/AlUNed.json",
                                         device=device)

In [ ]:
test_dataset = load_parallel(codelm_embedd_path="/kaggle/input/primevul-vuln-data/eatvul_primevul_covul/aggregated_codet5_embeddings.pt",
                             graph_embedd_path="/kaggle/input/primevul-vuln-data/eatvul_primevul_covul/n2v_node_embeddings.pt",
                             adjmat_path="/kaggle/input/primevul-vuln-data/eatvul_primevul_covul/adj_matrices.pt",
                             meta_path="/kaggle/input/primevul-vuln-data/eatvul_primevul_covul/AlUNed.json",
                             device=device)

In [ ]:
tokens_embedd_train1, graph_embedd_train1, adjmat_train1, label_train1 = train_benign1_dataset['train']
tokens_embedd_train2, graph_embedd_train2, adjmat_train2, label_train2 = train_benign2_dataset['train']
tokens_embedd_train3, graph_embedd_train3, adjmat_train3, label_train3 = train_vulnerable_dataset['train']
tokens_embedd_test, graph_embedd_test, adjmat_test, label_test = test_dataset['test']

In [ ]:
tokens_embedd_train = tokens_embedd_train1 + tokens_embedd_train2 + tokens_embedd_train3
graph_embedd_train = graph_embedd_train1 + graph_embedd_train2 + graph_embedd_train3
adjmat_train = adjmat_train1 + adjmat_train2 + adjmat_train3
label_train = label_train1 + label_train2 + label_train3

In [ ]:
# ----------------- Helpers -----------------
def _to_tensor(x, dtype=None, device=None):
    if isinstance(x, np.ndarray):
        t = torch.from_numpy(x)
    elif isinstance(x, torch.Tensor):
        t = x
    else:
        t = torch.tensor(x)
    if dtype is not None:
        t = t.to(dtype)
    if device is not None:
        t = t.to(device)
    return t

def _prepare_edge_index(edge_index):
    """ Ensure edge_index is LongTensor with shape [2, num_edges]. """
    if isinstance(edge_index, np.ndarray):
        ei = torch.from_numpy(edge_index)
    elif isinstance(edge_index, torch.Tensor):
        ei = edge_index
    else:
        ei = torch.tensor(edge_index)
    ei = ei.long()
    if ei.dim() == 2 and ei.shape[0] == 2:
        return ei.contiguous()
    if ei.dim() == 2 and ei.shape[1] == 2:
        return ei.t().contiguous()
    raise ValueError(f"edge_index has unsupported shape {ei.shape}")

def create_data_from_sample(graph_emb, edge_index, token_emb, label=None):
    x = _to_tensor(graph_emb, dtype=torch.float)
    ei = _prepare_edge_index(edge_index)
    tk = _to_tensor(token_emb, dtype=torch.float)
    data = Data(x=x, edge_index=ei, tokens=tk)
    if label is not None:
        data.y = _to_tensor(label, dtype=torch.long)
    return data

# ----------------- Class weight fallback -----------------
def _compute_class_weights(labels, device):
    """Fallback compute class weights if calculate_class_weights is not available.
       labels: 1D array-like integers"""
    labels = np.asarray(labels)
    classes, counts = np.unique(labels, return_counts=True)
    # weight = total / (num_classes * count)
    total = labels.shape[0]
    num_classes = classes.shape[0]
    weights = np.zeros(num_classes, dtype=np.float32)
    for i, c in enumerate(classes):
        weights[c] = total / (num_classes * counts[i])
    return torch.tensor(weights, dtype=torch.float, device=device)

# ----------------- Core: train single epoch (used inside fold) -----------------
def _train_one_epoch(model, loader, optimizer, loss_fn, device, aux_on=True, aux_weight=0.5, verbose=False):
    model.train()
    epoch_loss = 0.0
    total = 0
    all_preds = []
    all_labels = []
    pbar = tqdm(loader, desc="train", leave=False) if verbose else loader
    for batch in pbar:
        batch = batch.to(device)
        optimizer.zero_grad()
        xc_logis, xo_logis, xco_logis = model(batch, eval_random=True)
        if batch.y is None:
            raise ValueError("Batch item must contain .y")
        y = batch.y.view(-1).to(device)
        loss_main = loss_fn(xco_logis, y)
        if aux_on:
            loss_c = loss_fn(xc_logis, y)
            loss_o = loss_fn(xo_logis, y)
            loss = loss_main + aux_weight * (loss_c + loss_o)
        else:
            loss = loss_main
        loss.backward()
        optimizer.step()
        bs = y.size(0)
        epoch_loss += loss.item() * bs
        total += bs
        preds = xo_logis.argmax(dim=1).detach().cpu().numpy()
        labs = y.detach().cpu().numpy()
        all_preds.extend(preds.tolist())
        all_labels.extend(labs.tolist())
        if verbose:
            pbar.set_postfix({'batch_loss': loss.item()})
    avg_loss = epoch_loss / max(1, total)
    bal_acc = balanced_accuracy_score(all_labels, all_preds) if total>0 else 0.0
    return avg_loss, bal_acc

# ----------------- Core: evaluate on loader -----------------
def _eval_model(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            xc_logis, xo_logis, xco_logis = model(batch, eval_random=False)
            y = batch.y.view(-1).to(device)
            preds = xo_logis.argmax(dim=1).detach().cpu().numpy()
            labs = y.detach().cpu().numpy()
            all_preds.extend(preds.tolist())
            all_labels.extend(labs.tolist())
    return np.array(all_labels), np.array(all_preds)

def _save_confusion_matrix(y_true, y_pred, filename="confusion_matrix.png", labels=["Class 0", "Class 1"]):
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=labels, yticklabels=labels)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"📁 Saved confusion matrix to: {filename}")

# ----------------- Main k-fold training function -----------------
def train_kfold_causalgat(
    token_emb_list, graph_emb_list, edgeindex_list, label_list, device,
    n_splits=5,
    epochs=100,
    lr=1e-3,
    weight_decay=1e-3,
    batch_size=32,
    aux_weight=0.7,
    aux_on=True,
    patience=20,
    head=8,
    dropout=0.3,
    random_state=42,
    verbose=True
):
    """
    emb_list, edgeindex_list, label_list: parallel lists (or arrays)
    Returns: dict containing per-fold metrics and saved best model states
    """
    labels_np = np.asarray(label_list).astype(int)
    unique_labels = np.unique(labels_np)
    num_classes = int(unique_labels.max()) + 1

    # Choose average strategy for precision/recall:
    avg_strategy = 'binary' if num_classes == 2 else 'macro'

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    fold_results = []

    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels_np)), labels_np), start=1):
        if verbose:
            print(f"\n--- Fold {fold_idx}/{n_splits} ---")
        # Build train/val datasets
        train_dataset = [create_data_from_sample(graph_emb_list[i], edgeindex_list[i], token_emb_list[i], label_list[i]) for i in train_idx]
        val_dataset   = [create_data_from_sample(graph_emb_list[i], edgeindex_list[i], token_emb_list[i], label_list[i]) for i in val_idx]

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

        # compute class weights for this fold
        try:
            # try user-provided helper
            cw = calculate_class_weights([label_list[i] for i in train_idx], device)
            if not isinstance(cw, torch.Tensor):
                cw = torch.tensor(cw, dtype=torch.float, device=device)
        except Exception:
            cw = _compute_class_weights([label_list[i] for i in train_idx], device)

        # model init (infer num_features from first training sample)
        sample_feat = train_dataset[0].x.shape[1]
        model = CausalGAT(num_features=sample_feat, num_classes=num_classes, fusion_mode='cross_atten', head=head, dropout=dropout).to(device)

        loss_fn = nn.NLLLoss(weight=cw)
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val_score = -1.0  # we use balanced acc as selection criterion
        best_state = None
        best_epoch = -1
        no_imp = 0

        for epoch in range(1, epochs+1):
            # train one epoch
            train_loss, _ = _train_one_epoch(
                model, train_loader, optimizer, loss_fn, device,
                aux_on=aux_on, aux_weight=aux_weight, verbose=verbose
            )
            scheduler.step()

            # eval on validation
            y_val, y_pred = _eval_model(model, val_loader, device)
            prec = precision_score(y_val, y_pred, average=avg_strategy, zero_division=0)
            rec  = recall_score(y_val, y_pred, average=avg_strategy, zero_division=0)
            bal_acc = balanced_accuracy_score(y_val, y_pred)

            if verbose:
                print(f"Epoch {epoch:03d} | TrainLoss: {train_loss:.4f} | BalAcc: {bal_acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f}")

            # early stopping on balanced accuracy (you can change to other metric)
            if bal_acc > best_val_score:
                best_val_score = bal_acc
                best_state = copy.deepcopy(model.state_dict())
                best_epoch = epoch
                no_imp = 0
            else:
                no_imp += 1

            if no_imp >= patience:
                if verbose:
                    print(f"Early stopping (no improvement in {patience} epochs). Best val BalAcc={best_val_score:.4f} at epoch {best_epoch}")
                break

            gc.collect()
            torch.cuda.empty_cache()

        # After fold training, evaluate best model on val set
        if best_state is not None:
            torch.save(best_state, f"/kaggle/working/best_model_fold{fold_idx}.pt")
            model.load_state_dict(best_state)
        
        y_val, y_pred = _eval_model(model, val_loader, device)
        prec = precision_score(y_val, y_pred, average=avg_strategy, zero_division=0)
        rec  = recall_score(y_val, y_pred, average=avg_strategy, zero_division=0)
        bal_acc = balanced_accuracy_score(y_val, y_pred)

        if verbose:
            print(f"Fold {fold_idx} results -> BalAcc: {bal_acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

        fold_results.append({
            'fold': fold_idx,
            'bal_acc': bal_acc,
            'precision': prec,
            'recall': rec,
            'best_epoch': best_epoch
        })

    # aggregate metrics
    precisions = [f['precision'] for f in fold_results]
    recalls = [f['recall'] for f in fold_results]
    bal_accs = [f['bal_acc'] for f in fold_results]

    summary = {
        'folds': fold_results,
        'precision_mean': float(np.mean(precisions)),
        'precision_std': float(np.std(precisions)),
        'recall_mean': float(np.mean(recalls)),
        'recall_std': float(np.std(recalls)),
        'bal_acc_mean': float(np.mean(bal_accs)),
        'bal_acc_std': float(np.std(bal_accs))
    }

    if verbose:
        print("\n=== K-Fold Summary ===")
        print(f"Precision: {summary['precision_mean']:.4f} ± {summary['precision_std']:.4f}")
        print(f"Recall   : {summary['recall_mean']:.4f} ± {summary['recall_std']:.4f}")
        print(f"BalAcc   : {summary['bal_acc_mean']:.4f} ± {summary['bal_acc_std']:.4f}")

    return summary

In [ ]:
def test_kfold_causalgat(token_emb_list, graph_emb_list, edgeindex_list, label_list, device, kfold=5, best_state=None):
    """
    emb_list, edgeindex_list, label_list: parallel lists (or arrays)
    Returns: dict containing per-fold metrics and saved best model states
    """
    for fold_idx in range(1, kfold + 1):
        # Build test datasets
        test_dataset = [create_data_from_sample(graph_emb_list[i], edgeindex_list[i], token_emb_list[i], label_list[i]) for i in range(len(label_list))]
        test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

        # Evaluate best model on test set
        model = CausalGAT(num_features=200, num_classes=2, fusion_mode='cross_atten', head=8, dropout=0.5).to(device)
        if best_state is None:
            model.load_state_dict(torch.load(f"/kaggle/working/best_model_fold{fold_idx}.pt", map_location=device))
        else:
            model.load_state_dict(torch.load(best_state, map_location=device))
        
        y_val, y_pred = _eval_model(model, test_loader, device)
        _save_confusion_matrix(y_val, y_pred, f"/kaggle/working/fold{fold_idx}_results.png")

In [ ]:
train_kfold_causalgat(token_emb_list=tokens_embedd_train, graph_emb_list=graph_embedd_train, edgeindex_list=adjmat_train, label_list=label_train, device=device)

In [ ]:
test_kfold_causalgat(token_emb_list=tokens_embedd_test, graph_emb_list=graph_embedd_test, edgeindex_list=adjmat_test, label_list=label_test, device=device)